In [1]:
import ta
import pandas as pd
import numpy as np
import os 
import yfinance as yf

In [ ]:
benchmark_data = pd.read_csv('./data/djia.csv')

for i in [1,3,5,10,50,100]:
    benchmark_data['Return_'+str(i)+'d'] = np.log(benchmark_data['close']/benchmark_data['close'].shift(i))
benchmark_data.dropna(inplace=True)

for file in os.listdir('./data/raw'):
    if not file.endswith('.csv'):
        continue
    data = pd.read_csv('./data/raw/'+file)
    
    for i in [1,3,5,10,50,100]:
        data['Return_'+str(i)+'d'] = np.log(data['close']/data['close'].shift(i))
        
    data = data.merge(benchmark_data[['date','Return_1d','Return_3d','Return_5d','Return_10d','Return_50d','Return_100d']],on='date',how='left',suffixes=('_stock','_benchmark'))
        
    for i in [1,3,5,10,50,100]:
        data['Return_'+str(i)+'d_Correlation'] = data['Return_'+str(i)+'d_stock'].rolling(100).corr(data['Return_'+str(i)+'d_benchmark'])
        
    data['ADX'] = ta.trend.adx(high=data['high'],low=data['low'],close=data['close'])
    data['Aroon_Up'] = ta.trend.aroon_up(high=data['high'],low=data['low'])
    data['Aroon_Down'] = ta.trend.aroon_down(high=data['high'],low=data['low'])
    data['CCI'] = ta.trend.cci(high=data['high'],low=data['low'],close=data['close'])
    data['EMA'] = ta.trend.ema_indicator(close=data['close'])
    data['KAMA'] = ta.momentum.kama(close=data['close'])
    data['ROC'] = ta.momentum.roc(close=data['close'])
    data['RSI'] = ta.momentum.rsi(close=data['close'])
    data['CMF'] = ta.volume.chaikin_money_flow(high=data['high'],low=data['low'],close=data['close'],volume=data['volume'])
    data['ADI'] = ta.volume.acc_dist_index(high=data['high'],low=data['low'],close=data['close'],volume=data['volume'])
    data['FI'] = ta.volume.force_index(close=data['close'],volume=data['volume'])
    data['Bollinger_high'] = ta.volatility.bollinger_hband(close=data['close'])
    data['Bollinger_low'] = ta.volatility.bollinger_lband(close=data['close'])
    data['Donchian_low'] = ta.volatility.donchian_channel_lband(high=data['high'],low=data['low'],close=data['close'])
    data['Donchian_high'] = ta.volatility.donchian_channel_hband(high=data['high'],low=data['low'],close=data['close'])
    data.dropna(inplace=True)
    data = data.round(4)
    
    data = data.drop(columns=['Return_1d_benchmark','Return_3d_benchmark','Return_5d_benchmark','Return_10d_benchmark','Return_50d_benchmark','Return_100d_benchmark'])
    data.to_csv('./data/processed/'+file,index=False)

In [11]:
# download ^RUT

data = yf.download('^RUT',period='max',interval='1d').stack().reset_index()

data.columns = [x.lower() for x in data.columns]

data['date'] = data['date'].dt.strftime('%Y-%m-%d')
data.drop(columns=['adj close','ticker'],inplace=True)
data = data.round(4)
data.to_csv('./data/russell.csv',index=False)

[*********************100%***********************]  1 of 1 completed
C:\Users\Dell\AppData\Local\Temp\ipykernel_12504\151345898.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = yf.download('^RUT',period='max',interval='1d').stack().reset_index()
